In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings

from gsheet import *
from mailer import *
from payment_info import *
from qr_code import *
from dataclasses import dataclass
from payment_data import *
import pandas as pd
import dataclasses
import logging

Succesfully read config from: ['./config.cfg']


In [3]:
logging.basicConfig(
    format="[%(levelname)s] %(message)s",
    level=logging.INFO
)

In [4]:
sheet = Sheets(SPREADSHEET_ID)

[INFO] file_cache is only supported with oauth2client<4.0.0


In [6]:
# sheet_names = sheet.get_sheet_names()
sheet_name = '4pvs_all'
if sheet_name=='TEST':
    testmode=True
else:
    testmode=False
df = sheet.get_dataframe(sheet_name)

In [7]:
pdf=PaymentsDataFrame(df, NEEDED_COLS)

[INFO] Of 233 people, missing email: 8,
	emailable paid: 189
	emailable unpaid: 36.
 Also there were 0 invalid emails.


In [8]:
SENDER="4pvs@skaut.cz"
SENDER_NAME="4. přístav Jana Nerudy"

In [13]:
emails = []
for i, row in pdf.df_emailable_unpaid.iterrows():
    recepient_addresses = row.valid_emails
    pi = PaymentInfo.from_df_row(row)
    emails.append(PaymentEmail(pi, SENDER, SENDER_NAME, recepient_addresses, testmode=testmode).create_email())

In [14]:
print(emails[1])

From: "4. pristav Jana Nerudy" <4pvs@skaut.cz>
To: "Matka: mail" <jitacihi@gmail.com>
Subject: =?utf-8?q?4=2e=20p=c5=99=c3=adstav=20Jana=20Nerudy=20-=20p=c5=99=c3=adsp=c4=9bvky=20podzim=202021=20=282=2e=20v=c3=bdzva=20k=20platb=c4=9b=29?=
Cc: "Nemo" <buzek.michal@centrum.cz>
Date: Sat, 20 Nov 2021 15:39:21 +0000
Message-Id: <emdb07f6a8-3b97-48c2-b6d7-bf28d321ffcb@escarabajo>
Reply-To: "4. pristav Jana Nerudy" <4pvs@skaut.cz>
User-Agent: PaymentMailer/0.1
Mime-Version: 1.0
Content-Type: multipart/related; boundary="------=_MBC05D33A5-1273-4469-9734-7D78117472D2"

--------=_MBC05D33A5-1273-4469-9734-7D78117472D2
Content-Type: multipart/alternative;
 boundary="------=_MB462F58BD-D8F2-4297-91B9-95D09C74EA90"

--------=_MB462F58BD-D8F2-4297-91B9-95D09C74EA90
Content-Type: text/plain; charset=utf-8; format=flowed
Content-Transfer-Encoding: quoted-printable

Dobr=C3=BD den / Ahoj,
od tohoto =C5=A1koln=C3=ADho roku se sna=C5=BE=C3=ADme vyb=C3=ADrat p=C5=99=
=C3=ADsp=C4=9Bvky centralizovan=C4=9

In [11]:
mailer = Mailer()

[INFO] file_cache is only supported with oauth2client<4.0.0


In [15]:
sent = []
for email in emails[1:]:
    msg = mailer.send_message(SENDER, email)
    sent.append(msg)

[INFO] Message Id: 17d3fdee4c2b0580 sent.
[INFO] Message Id: 17d3fdee6c452dd7 sent.
[INFO] Message Id: 17d3fdee8b3c0470 sent.
[INFO] Message Id: 17d3fdeebf036b5a sent.
[INFO] Message Id: 17d3fdeedc051436 sent.
[INFO] Message Id: 17d3fdeef71c9c89 sent.
[INFO] Message Id: 17d3fdef06d003e4 sent.
[INFO] Message Id: 17d3fdef25eaaca4 sent.
[INFO] Message Id: 17d3fdef5c2f3500 sent.
[INFO] Message Id: 17d3fdef7af6e96a sent.
[INFO] Message Id: 17d3fdef9eeae5fa sent.
[INFO] Message Id: 17d3fdefad6fa08d sent.
[INFO] Message Id: 17d3fdefddfd584c sent.
[INFO] Message Id: 17d3fdefe1f37116 sent.
[INFO] Message Id: 17d3fdf00c4a5971 sent.
[INFO] Message Id: 17d3fdf02e1ae090 sent.
[INFO] Message Id: 17d3fdf05e00e84a sent.
[INFO] Message Id: 17d3fdf063d9f07b sent.
[INFO] Message Id: 17d3fdf0844b883b sent.
[INFO] Message Id: 17d3fdf0a879d82a sent.
[INFO] Message Id: 17d3fdf0cccfa684 sent.
[INFO] Message Id: 17d3fdf0e713f4f9 sent.
[INFO] Message Id: 17d3fdf11c3ed01d sent.
[INFO] Message Id: 17d3fdf12b6af79